In [48]:
from flask import Flask, render_template, request
import pandas as pd
from flask import jsonify
from flask_cors import CORS

app = Flask(__name__, static_url_path='/static', static_folder='static')

In [49]:
# _________________________________________________________________________________________________________________________
# Specify the path to your Excel file
excel_file_path = '/Users/abbywoolf/Desktop/Costing-Tool/Cleaned_Cost_Data.xlsx'
# /Users/abbywoolf/Desktop/Costing-Tool
# Specify the sheet name you want to import
sheet_name = 'Frames'  # Replace 'Sheet1' with the actual sheet name
# Read the Excel file into a DataFrame
df = pd.read_excel(excel_file_path, sheet_name=sheet_name)
# __________________________________________________________________________________________________________________________-

In [50]:
# MAKE Frames_df
frames_df = df.dropna()
# frames_df['Frame'] = frames_df['Frame'].str.lower()
frames_df.loc[:, 'Frame'] = frames_df['Frame'].str.lower()
# frames_df['GP Price'] = frames_df['GP Price'].astype(float)

printer_data = {
    'PrinterName': ['inca-reductive', 'inca-synograph', 'epson', 'swissQ-synograph', ' SwissQ-Reductive', 'canon'],
    'labor': [8.0, 8.0, 5.0, 20.0, 20.00, 7.0],
    'ink': [.08, 1.68, 0.22, 2.80, .22, .27]
}
printer_costs_df = pd.DataFrame(printer_data)
# Make substrate df
substrate_data = {
    'SubstrateType': ['canvas', 'Epson-paper', 'Canon-paper', 'mdf', 'tar-board'],
    'Price/sf': [.66, 1.35, 1.35, 0.99, .17]
}
substrate_df = pd.DataFrame(substrate_data)

# Make hardware df
hardware_data = {
    'hardwareType': ['cleat', 'wire', 'd-rings'],
    'price': [5.00, 2.00, 0]
}
hardware_df = pd.DataFrame(hardware_data)

In [51]:
# ___________________________FUNCTION______________________________________________
def total_cost(length, width, frame_type, substrate, printer, hardware):
    # valid_options_arg1 = ['fps','gwt','bfl','cfl','gfl','bgf','nfl','wfl','bcap','ncap','gcap','walcap']
    # valid_options_arg2 = ['canvas', 'Epson-paper', 'Canon-paper', 'mdf']
    # valid_options_arg3 = ['inca-reductive', 'inca-synograph', 'epson', 'swissQ-synograph', ' SwissQ-Reductive', 'canon']
    # valid_options_arg4 = ['cleat', 'wire', 'd-rings']

    # # Check if the arguments are in the valid options
    # if frame_type not in valid_options_arg1:
    #     raise ValueError(f"Invalid value for arg1. Allowed options are: {valid_options_arg1}")
    # if substrate not in valid_options_arg2:
    #     raise ValueError(f"Invalid value for arg2. Allowed options are: {valid_options_arg2}")
    # if printer not in valid_options_arg3:
    #     raise ValueError(f"Invalid value for arg3. Allowed options are: {valid_options_arg3}")
    # if hardware not in valid_options_arg4:
    #     raise ValueError(f"Invalid value for arg4. Allowed options are: {valid_options_arg4}")
    
  #_______________________________________________________________________________ 
    # Calc Substrate Costs
    price_per_sf = substrate_df.loc[substrate_df['SubstrateType'] == substrate, 'Price/sf'].values[0]
    if substrate == 'canvas':
        if length > 58 and width > 58:
            cost_of_substrate = (((length + 11) * (width + 11)) / 144) * price_per_sf
        else:
            cost_of_substrate = (((length + 8) * (width + 8)) / 144) * price_per_sf

    elif substrate == 'Epson-paper' or 'Canon-paper':
        cost_of_substrate = (((length + 1.5) * (width +1.5)) / 144) * price_per_sf

    elif substrate == 'mdf':
        #Check if dimensions match 49x60
        if (length == 60 and width == 48) or (length == 48 and width == 60):
            cost_of_substrate= 15.32
        else:
            cost_of_substrate = ((length * width) / 144) * 1.2
    else:
        raise ValueError("Invalid substrate")
    #return cost_of_substrate

    # price_per_sf = substrate_df.loc[substrate_df['Substrate'] == substrate, 'Price/sf'].values[0]
    #cost_of_substrate = square_feet * price_per_sf
    
  #_______________________________________________________________________________  
    # Calc Frame costs
    # Check if a matching row exists in the DataFrame
    length= float(length)
    width= float(width)
    matching_rows = frames_df[
        (frames_df['Length'] == length) &
        (frames_df['Width'] == width) &
        (frames_df['Frame'].str.contains(frame_type))
    ]

    if not matching_rows.empty:
        cost_of_frame = matching_rows['GP Price'].values[0]
        return cost_of_frame
    else:
        print(f"Frame with Length {length}, Width {width}, and Type {frame_type} not found.")
        return None  # Or any other appropriate value or indication

#__________________________________________________________________________________
    # Implement the calculation for printer costs

    ink_per_sf = printer_costs_df.loc[printer_costs_df['PrinterName'] == printer, 'ink'].values[0]
    labor_cost = printer_costs_df.loc[printer_costs_df['PrinterName'] == printer, 'labor'].values[0]

    total_printer_cost = ((ink_per_sf) * (length * width) / 144) + labor_cost
#_______________________________________________________________________________ 
    #implement hardware costs
    hardware_cost = hardware_df.loc[hardware_df['hardwareType'] == hardware, 'price'].values[0]
#___________________________________________________________________________________
    # Create the desired margin calculator
    final_cost = cost_of_substrate + cost_of_frame + total_printer_cost + hardware_cost
    return final_cost

 #___________________________________________________________________________________   
    # margin_dec = margin_percent * .01
    # royality_dec = royality_percent * .01
    
    # wsp = final_cost_no_roy/(1-royality_dec - margin_dec)
    # royalty_value = wsp * royality_dec
    # final_cost_with_roy = final_cost_no_roy + royalty_value
    # return final_cost_with_roy
 #________________________END FUNCTION______________________________________________     

In [52]:
test = total_cost(60, 60, 'bfl', 'canvas', 'inca-synograph','cleat')
test

265

In [53]:
def test_substrate_cost(length, width, substrate):
    price_per_sf = substrate_df.loc[substrate_df['SubstrateType'] == substrate, 'Price/sf'].values[0]
    if substrate == 'canvas':
        if length > 58 and width > 58:
            cost_of_substrate = (((length + 11) * (width + 11)) / 144) * price_per_sf
        else:
            cost_of_substrate = (((length + 8) * (width + 8)) / 144) * price_per_sf

    elif substrate == 'Epson-paper' or substrate == 'Canon-paper':
        cost_of_substrate = (((length + 1.5) * (width +1.5)) / 144) * price_per_sf


    elif substrate == 'tar-board':
        cost_of_substrate = (((length + .125) * (width + .125)) / 144) * price_per_sf
    
    elif substrate == 'mdf':
        #Check if dimensions match 49x60
        if (length == 60 and width == 48) or (length == 48 and width == 60):
            cost_of_substrate= 15.32
        else:
            cost_of_substrate = (((length * width) / 144) * 1.2) * price_per_sf
    else:
        raise ValueError("Invalid substrate")


    return cost_of_substrate

In [54]:
test_substrate = test_substrate_cost(60, 60, 'canvas')
test_substrate

23.104583333333334

# The larger dimension needs to be the first argument

In [55]:
def test_frame_cost(length, width, frame_type):
    # Calc Frame costs
    # Check if a matching row exists in the DataFrame
    length= float(length)
    width= float(width)
    matching_rows = frames_df[
        (frames_df['Length'] == length) &
        (frames_df['Width'] == width) &
        (frames_df['Frame'].str.contains(frame_type))
    ]

    if not matching_rows.empty:
        cost_of_frame = matching_rows['GP Price'].values[0]
        return cost_of_frame
    else:
        print(f"Frame with Length {length}, Width {width}, and Type {frame_type} not found.")
        return None  # Or any other appropriate value or indication

In [56]:
test_frames = test_frame_cost(60,60,'bfl')
test_frames

265

# Doesn't include printing on wrap around portion

In [57]:
    # Implement the calculation for printer costs
def test_printer_cost(length, width, printer):
    
        ink_per_sf = printer_costs_df.loc[printer_costs_df['PrinterName'] == printer, 'ink'].values[0]
        labor_cost = printer_costs_df.loc[printer_costs_df['PrinterName'] == printer, 'labor'].values[0]

        total_printer_cost = ((ink_per_sf) * (length * width) / 144) + labor_cost
        return total_printer_cost

In [58]:
test_printer = test_printer_cost(60,60,'inca-synograph')
test_printer

50.0

In [59]:
def test_hardware_cost(hardware):
#implement hardware costs
    hardware_cost = hardware_df.loc[hardware_df['hardwareType'] == hardware, 'price'].values[0]
    return hardware_cost

In [60]:
test_hardware = test_hardware_cost('cleat')
test_hardware

5.0

In [61]:
    # Create the desired margin calculator
def royalty_calculator(final_cost, margin_percent, royalty_percent):


 #___________________________________________________________________________________   
        margin_dec = margin_percent * .01
        royalty_dec = royalty_percent * .01
    
        wsp = final_cost/(1-royalty_dec - margin_dec)
        royalty_value = wsp * royalty_dec
        final_cost_with_roy = final_cost + royalty_value
        return final_cost_with_roy
 #________________________END FUNCTION______________________________________________ 

In [62]:
test_royalty = royalty_calculator(343, 65, 5)
test_royalty

400.1666666666667

In [63]:
    # Create the desired margin calculator
def wsp_calculator(final_cost, margin_percent, royalty_percent):
 #___________________________________________________________________________________   
        margin_dec = margin_percent * .01
        royalty_dec = royalty_percent * .01
    
        wsp = final_cost/(1-royalty_dec - margin_dec)
        royalty_value = wsp * royalty_dec
        final_cost_with_roy = final_cost + royalty_value
        return wsp
 #________________________END FUNCTION______________________________________________ 

In [64]:
test_wsp = wsp_calculator(343, 65, 5)
test_wsp

1143.3333333333335

In [65]:
# def final_cost(length, width, frame_type, printer, substrate, hardware):
#     test_substrate = test_substrate_cost(60, 60, 'canvas')
#     test_frames = test_frame_cost(60,60,'bfl')
#     test_printer = test_printer_cost(60,60,'inca-synograph')
#     test_hardware = test_hardware_cost('cleat')
#     test_wsp = wsp_calculator(343, 65, 5)
#     final_cost = test_substrate + test_frames + test_printer + test_hardware
#     cost_with_roy = margin_calculator(final_cost, 65, 5)

#     data = {'Substrate Cost': [test_substrate],
#             'Frame Cost': [test_frames],
#             'Printer Costs': [test_printer],
#             'Harware Costs': [test_hardware],
#             'Final GP Cost': [final_cost],
#             'Cost w/ Royalty': [cost_with_roy],
#             'WSP @ % Margin': [test_wsp]}
#     results_df = pd.DataFrame(data)
#     return results_df
    

In [66]:
# test_final_cost = final_cost(60,60,'bfl','inca-synograph','canvas','cleat')
# test_final_cost 

,Substrate Cost,Frame Cost,Printer Costs,Harware Costs,Final GP Cost,Cost w/ Royalty,WSP @ % Margin
0,23.104583,265,50.0,5.0,343.104583,400.288681,1143.333333


In [67]:
def final_cost(length, width, frame_type, printer, substrate, hardware, margin_percent, royalty_percent):
    test_substrate = test_substrate_cost(length, width, substrate)
    test_frames = test_frame_cost(length, width,frame_type)
    test_printer = test_printer_cost(length, width,printer)
    test_hardware = test_hardware_cost(hardware)
    
    final_cost = test_substrate + test_frames + test_printer + test_hardware
    test_wsp = wsp_calculator(final_cost, margin_percent, royalty_percent)
    cost_with_roy = margin_calculator(final_cost, margin_percent, royalty_percent)

    data = {'Substrate Cost': [test_substrate],
            'Frame Cost': [test_frames],
            'Printer Costs': [test_printer],
            'Harware Costs': [test_hardware],
            'Final GP Cost': [final_cost],
            'Cost w/ Royalty': [cost_with_roy],
            'WSP @ % Margin': [test_wsp]}
    results_df = pd.DataFrame(data)
    return results_df

In [68]:
test_final_cost = final_cost(60,60,'bfl','inca-synograph','canvas','cleat', 65, 5)
test_final_cost 

,Substrate Cost,Frame Cost,Printer Costs,Harware Costs,Final GP Cost,Cost w/ Royalty,WSP @ % Margin
0,23.104583,265,50.0,5.0,343.104583,400.288681,1143.681944


# Trying to combine all little functions into 1 big function

In [46]:
def test_combined(length, width, frame_type, printer, substrate, hardware, margin_percent, royalty_percent):
    price_per_sf = substrate_df.loc[substrate_df['SubstrateType'] == substrate, 'Price/sf'].values[0]
    if substrate == 'canvas':
        if length > 58 and width > 58:
            cost_of_substrate = (((length + 11) * (width + 11)) / 144) * price_per_sf
        else:
            cost_of_substrate = (((length + 8) * (width + 8)) / 144) * price_per_sf

    elif substrate == 'Epson-paper' or substrate == 'Canon-paper':
        cost_of_substrate = (((length + 1.5) * (width +1.5)) / 144) * price_per_sf


    elif substrate == 'tar-board':
        cost_of_substrate = (((length + .125) * (width + .125)) / 144) * price_per_sf
    
    elif substrate == 'mdf':
        #Check if dimensions match 49x60
        if (length == 60 and width == 48) or (length == 48 and width == 60):
            cost_of_substrate= 15.32
        else:
            cost_of_substrate = (((length * width) / 144) * 1.2) * price_per_sf
    else:
        raise ValueError("Invalid substrate")
#_____________________________________________________
 
    # Calc Frame costs
    # Check if a matching row exists in the DataFrame
    length= float(length)
    width= float(width)
    matching_rows = frames_df[
        (frames_df['Length'] == length) &
        (frames_df['Width'] == width) &
        (frames_df['Frame'].str.contains(frame_type))
    ]

    if not matching_rows.empty:
        cost_of_frame = matching_rows['GP Price'].values[0]
    else:
        print(f"Frame with Length {length}, Width {width}, and Type {frame_type} not found.")
        return None  # Or any other appropriate value or indication

#_____________________________________________________
    # Implement the calculation for printer costs
    
        ink_per_sf = printer_costs_df.loc[printer_costs_df['PrinterName'] == printer, 'ink'].values[0]
        labor_cost = printer_costs_df.loc[printer_costs_df['PrinterName'] == printer, 'labor'].values[0]
        total_printer_cost = ((ink_per_sf) * (length * width) / 144) + labor_cost
 #_____________________________________________________
        hardware_cost = hardware_df.loc[hardware_df['hardwareType'] == hardware, 'price'].values[0]
 #_____________________________________________________
        # Create the desired margin calculator
 
        margin_dec = margin_percent * .01
        royalty_dec = royalty_percent * .01
        final_GP_cost = cost_of_substrate+cost_of_frame+total_printer_cost+hardware_cost
        wsp = final_GP_cost/(1-royalty_dec - margin_dec)
        royalty_value = wsp * royalty_dec
        final_cost_with_roy = final_GP_cost + royalty_value
 #________________________END FUNCTION______________________________________________ 


    data = {'Substrate Cost': [cost_of_substrate],
            'Frame Cost': [cost_of_frame],
            'Printer Costs': [total_printer_cost],
            'Harware Costs': [hardware_cost],
            'Final GP Cost': [final_GP_cost],
            'Cost w/ Royalty': [final_cost_with_roy],
            'WSP @ % Margin': [wsp]}
    results_df = pd.DataFrame(data)
    return results_df

In [47]:
test_combined = test_combined(60,60,'bfl','inca-synograph','canvas','cleat', 65, 5)
test_combined

UnboundLocalError: cannot access local variable 'total_printer_cost' where it is not associated with a value